# Compare two ROI files and return the score and the ROIs of False Positives and False Negatives detection

Compare two ROI .zip files, one containing manual annotation of cell event, and the one containing the DeXtrusion detection, and print the accuracy scores. It can also generate ROI .zip file containing the False Positives and False Negatives (of DeXtrusion results compared to manual one) files to allow to check them.

In [ ]:
# default parameters
talkative = True  ## print info messages
import os
from glob import glob
import numpy as np
imname = os.getcwd()
# DeXNet path, to read the configuration file
modeldir = "./deXNets/notum_all/notum_all_original/notumAll0_retrained0907"
from dextrusion.DeXtrusion import DeXtrusion
dexter = DeXtrusion(verbose=talkative)

Tensorflow with Cuda: False
Tensorflow version: 2.15.0
Num GPUs Available:  0


In [12]:
#### Parameters to choose

dxy = 15          ## distance to ground-truth ROI to consider same
dt = 4            ## distance in time to ground-truth ROI to consider same
cat = 1           ## event to score (here cell_death)
catnames = ["", "_cell_death.zip", "_cell_sop.zip", "_cell_division.zip"]

## path the ROIs files are
#test_path = "../../data/notum_retrain_mix/test/"
test_path = "/Users/yuyangsmacbook/Desktop/"
## path the results folder where DeXtrusion ROI files are
res = test_path+"results/"

# parameters used to create the ROIs (to keep in memory in the score datasheet)
volume_thres = 800
prob_thres = 180

In [ ]:
## parameter to consider ROIs as the same within DeXtrusion (spatial and temporal distance threshold)
distxy=10
dist=4

### Write results to file
resultfile = res+"/score.csv"
header = 'Name;VolumeThreshold;ProbaThreshold;distanceXY;distanceT;TP;FP;FN;Precision;Recall' 

## Test all the files present in the result folder
resfiles = glob(res+"/*"+catnames[cat], recursive = False)
resarray = None

for resfile in resfiles:
    testname = os.path.basename(resfile)
    ind = testname.find(catnames[cat])
    testname = testname[0:ind]
    print(testname)
    paras = np.array([testname, volume_thres, prob_thres, dxy, dt])
    outfile = res+testname+catnames[cat]
    #dexter.get_rois_of_scaled_image( res+testname+catnames[cat]+"_proba.tif", volume_thres, prob_thres, distxy, dist, cat=cat, outfile=outfile)
    
    # compare the ROIs and print the scores
    score = dexter.compare_rois(cat=cat, gtroisfile=test_path+testname+catnames[cat], resroisfile=outfile, distance_xy=dxy, distance_t=dt)
    
    # write in .zip files the false detections (put # in the beginning of the lines to not do this step)
    dexter.write_falsepositives(cat=cat, resfile=res+testname+catnames[cat], gtfile=test_path+testname+catnames[cat], distance_xy=dxy, distance_t=dt)
    dexter.write_falsenegatives(cat=cat, resfile=res+testname+catnames[cat], gtfile=test_path+testname+catnames[cat],distance_xy=dxy, distance_t=dt)
    
    array = np.atleast_2d(np.append(paras, np.array(score)))
    if resarray is None:
        resarray = array
    else:
        resarray = np.append(resarray, array, axis=0)

## save the scores in a tabular file
with open(resultfile,'w') as csvfile:
    np.savetxt(csvfile, resarray, delimiter=';', header=header, fmt='%s', comments='')

ValueError: Expected 1D or 2D array, got 0D array instead

Mix two proba_tif

In [4]:
import os, glob
import numpy as np
import tifffile as tiff

# ====== 配置 ======
# 事件类别：按需选择 1..(len(catnames)-1)，比如 [1] 只融合 cell_death
CATS = [1,2,3]   # 1: _cell_death, 2: _cell_sop, 3: _cell_division

catnames = ["", "_cell_death.zip", "_cell_sop.zip", "_cell_division.zip"]

# 多个来源结果目录（目录内应该有 *_<cat>_proba.tif）
RES_DIRS = [
    "/Users/yuyangsmacbook/Desktop/results/retrain466_n2",
    "/Users/yuyangsmacbook/Desktop/results/retrain466_nn2"
    #"/Users/yuyangsmacbook/Desktop/results/retrain466_4",
    #"/Users/yuyangsmacbook/Desktop/results/retrain466_5"
]
# 融合输出目录
OUT_DIR = "/Users/yuyangsmacbook/Desktop/results"
os.makedirs(OUT_DIR, exist_ok=True)

# 融合策略：'mean' | 'max' | 'weighted'
ENSEMBLE_MODE = 'mean'
# 若使用加权平均，请给出与 RES_DIRS 等长的权重
WEIGHTS = None  # 例如 [0.4, 0.6]；或保持 None

# 要求至少多少个来源同时存在该样本才融合（缺失时跳过）
MIN_K = len(RES_DIRS)  # 设为 len(RES_DIRS) 表示所有来源都要有

# ====== 工具函数 ======
def pattern_for_cat(cat):
    return "*" + catnames[cat].replace(".zip","") + "_proba.tif"

def collect_single_dir(res_dir, pattern):
    m = {}
    for p in glob.glob(os.path.join(res_dir, pattern)):
        base = os.path.basename(p).replace("_proba.tif","")  # e.g. A_cell_death
        m[base] = p
    return m

def cast_back(avg, ref_dtype):
    if np.issubdtype(ref_dtype, np.integer):
        info = np.iinfo(ref_dtype)
        avg = np.clip(avg, info.min, info.max).astype(ref_dtype)
    else:
        avg = avg.astype(ref_dtype)
    return avg

# ====== 主流程 ======
total_written = 0
for cat in CATS:
    pat = pattern_for_cat(cat)
    # 收集每个目录的 {base -> path}
    maps_per_dir = [collect_single_dir(d, pat) for d in RES_DIRS]
    # 交集或“至少出现 MIN_K 个来源”的键集合
    all_keys = set().union(*[set(m.keys()) for m in maps_per_dir])
    keys = [k for k in all_keys if sum(k in m for m in maps_per_dir) >= MIN_K]
    keys.sort()

    print(f"[cat={cat} {catnames[cat]}] candidates: {len(all_keys)}, to fuse: {len(keys)}")

    for base in keys:
        paths = [m[base] for m in maps_per_dir if base in m]

        # 读入所有存在的图
        ims = []
        for p in paths:
            im = tiff.imread(p)
            ims.append(im)

        # 尺寸校验（必须一致）
        shapes = {im.shape for im in ims}
        if len(shapes) != 1:
            print("  Skip (shape mismatch):", base, shapes)
            continue

        ref_dtype = ims[0].dtype
        stack = np.stack([im.astype(np.float32) for im in ims], axis=0)

        # 融合
        if ENSEMBLE_MODE == 'max':
            fused = np.max(stack, axis=0)
        elif ENSEMBLE_MODE == 'weighted':
            if WEIGHTS is None or len(WEIGHTS) != len(ims):
                print("  Skip (bad WEIGHTS):", base)
                continue
            w = np.asarray(WEIGHTS, dtype=np.float32)
            w = w / np.sum(w)
            fused = np.tensordot(w, stack, axes=(0,0))  # 加权和
        else:  # 'mean'
            fused = np.mean(stack, axis=0)

        fused = cast_back(fused, ref_dtype)

        # 写出：保持命名 <base>_proba.tif
        outp = os.path.join(OUT_DIR, base + "_proba.tif")
        tiff.imwrite(outp, fused, imagej=True)
        total_written += 1
        print("  Wrote:", outp)

print("Done. total fused:", total_written)
print("Outputs in:", OUT_DIR)


[cat=1 _cell_death.zip] candidates: 1, to fuse: 1
  Wrote: /Users/yuyangsmacbook/Desktop/results/img_0001_0050_center_2048x1024_cell_death_proba.tif
[cat=2 _cell_sop.zip] candidates: 1, to fuse: 1
  Wrote: /Users/yuyangsmacbook/Desktop/results/img_0001_0050_center_2048x1024_cell_sop_proba.tif
[cat=3 _cell_division.zip] candidates: 1, to fuse: 1
  Wrote: /Users/yuyangsmacbook/Desktop/results/img_0001_0050_center_2048x1024_cell_division_proba.tif
Done. total fused: 3
Outputs in: /Users/yuyangsmacbook/Desktop/results
